In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [8]:
def prob_news(K, n):
    """Create the problem in cvxpy
    Parameters
    ----------
    K: int
        Number of data samples (clusters)
    n: int
        Number of products
    Returns
    -------
    The instance and parameters of the cvxpy problem
    """
    d_train = cp.Parameter((K, 2))
    x = cp.Variable(2)
    s = cp.Variable(K)
    buy_p = cp.Parameter(2)
    sell_p = cp.Parameter(2)
    wk = cp.Parameter(K)
    lam = cp.Variable()
    eps = cp.Parameter()
    C_r = np.vstack([-np.eye(2), np.eye(2)])
    d_r = np.hstack([np.zeros(2), np.ones(2)*40])
    gam = cp.Variable((4, K*4))

    objective = cp.Minimize(buy_p@x + eps*lam + s@wk)
    constraints = []
    # formulate constraints

    for k in range(K):
        constraints += [-sell_p@x
                        + gam[0, (k*4):((k+1)*4)]@(
                            d_r - C_r@d_train[k]) <= s[k]]
        constraints += [-sell_p[0]*x[0] - d_train[k] @
                        (sell_p[1]*np.array([0, 1]))
                        + gam[1, (k*4):((k+1)*4)]@(d_r
                                                   - C_r@d_train[k]) <= s[k]]
        constraints += [-d_train[k]@(sell_p[0]*np.array([1, 0]))
                        - sell_p[1]*x[1] +
                        gam[2, (k*4):((k+1)*4)]@(d_r - C_r@d_train[k]) <= s[k]]
        constraints += [-d_train[k]@(sell_p) +
                        gam[3, (k*4):((k+1)*4)]@(d_r - C_r@d_train[k]) <= s[k]]
        constraints += [cp.norm(C_r.T@gam[0, (k*4):((k+1)*4)],1) <= lam]
        constraints += [cp.norm(C_r.T@gam[1, (k*4):((k+1)*4)] +
                                sell_p[1]*np.array([0, 1]), 1) <= lam]
        constraints += [cp.norm(C_r.T@gam[2, (k*4):((k+1)*4)] +
                                sell_p[0]*np.array([1, 0]), 1) <= lam]
        constraints += [cp.norm(C_r.T@gam[3, (k*4):((k+1)*4)] +
                                sell_p, 1) <= lam]

    constraints += [x >= 0, gam >= 0]

    problem = cp.Problem(objective, constraints)

    return problem, x, s, lam, d_train, wk, eps, gam, buy_p, sell_p

In [9]:
def gen_demand(N, R, seed):
    """ Generates log-normal demand, clipped to be within polyhedral"""
    np.random.seed(seed)
    sig = np.array([[0.3, -0.1], [-0.1, 0.2]])
    mu = np.array((3, 2.8))
    norms = np.random.multivariate_normal(mu, sig, (N, R))
    d_train = np.exp(norms)
    d_train = np.minimum(d_train, 40)
    # d_train = np.round(d_train)
    return d_train

In [10]:
def cluster_data(D_in, K):
    """Return K cluster means after clustering D_in into K clusters
    Parameters
    ----------
    D_in: array
        Input dataset, N entries
    Returns
    -------
    Dbar_in: array
        Output dataset, K entries
    weights: vector
        Vector of weights for Dbar_in
    """
    N = D_in.shape[0]
    kmeans = KMeans(n_clusters=K,n_init = 'auto').fit(D_in)
    Dbar_in = kmeans.cluster_centers_
    weights = np.bincount(kmeans.labels_) / N

    return Dbar_in, weights, kmeans

In [5]:
def evaluate(buy_p, sell_p, x, d):
    """ Calculates the out of sample expected value
    Parameters
    ----------
    buy_p: vector
        Buying Prices
    sell_p: vector
        Selling Prices
    x: vector
        Decision variables of the optimization problem
    d: matrix
        Validation data matrix
    Returns:
    -------
    float: out of sample expected value
    """
    totval = 0
    totval = np.maximum(-sell_p[0]*x.value[0], -sell_p[0]*d[:, 0]) + \
        np.maximum(-sell_p[1]*x.value[1], -sell_p[1]*d[:, 1])
    return buy_p@x.value + np.mean(totval)

In [11]:
N_tot = 100 #Total number of data-points
M = 10
R = 30     # Total times we repeat experiment to estimate final probabilty
n = 2  # number of products
eps_nums = np.linspace(0.01,3,30)

buy_pval = np.array([4., 5.])
sell_pval = np.array([5, 6.5])
C_r = np.vstack([-np.eye(2), np.eye(2)])

Data = gen_demand(N_tot, R, 2)
Data_eval = gen_demand(N_tot, R, 3)

In [7]:
K = 10 # number of clusters
r = 0 # current iteration
eps = 0.1 #current epsilon

dat_orig = Data[:, r, :]
d_train, wk, kmeans = cluster_data(dat_orig, K)
dat_eval = Data_eval[:, r, :]
problem, x, s, lam, data_train_pm, w_pm, eps_pm, gam, buy_p, sell_p = prob_news(K, n)
data_train_pm.value = d_train
w_pm.value = wk
buy_p.value = buy_pval
sell_p.value = sell_pval
eps_pm.value = eps
problem.solve(solver = cp.CLARABEL)
evalvalue = evaluate(buy_pval, sell_pval, x, dat_eval) 
satisfy = float(evalvalue <= problem.objective.value) 
print("DRO objective value :", problem.objective.value)
print("Out-of-sample expected value :", evalvalue)
print("Boolean for constraint satisfaction: ", satisfy)

TypeError: '<=' not supported between instances of 'str' and 'int'